In [143]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import re

## Flight Delay Prediction EDA: 
In order to predict the delay and cancellation of flights we will analyze the flight delay and cancellation dataset that has flight information on 2015. In order to understand better the data given about flights we will analyze first each dataset and then we will make a united analysis of the data. 

The EDA will be structured in the following manner: 
<style>
  ol {
    list-style-type: none;
  }
  ol li:before {
    content: attr(value)". ";
  }
</style>

<ol>
  <li>Explain the structure of the data</li>
  <li>Analyze each table in order to understand if it has any missing values or inconsistencies.</li>
  <li>Unite the data into one DataFrame.</li>
  <li>
    Explore the data in order to find some interesting insights. In this section we will try to answer some important questions:
    <ol type="a">
      <li>How is the distribution of delayed flights and non-delayed flights?</li>
      <li>Is there any flight route that has more delays?</li>
      <li>How are the airlines flights distributed in the dataset?</li>
      <li>Is there any country with more delays?</li>
    </ol>
  </li>
 <li>Feature Engineering</li>
 <li>Data Preparation </li>
  <li>Training model</li>
</ol>





# Loading Data: 

In [144]:
## Setting paths: 
dataBaseDir = os.path.join("..","GeneralData",)
airline_path = os.path.join(dataBaseDir,  "airlines.csv")
airports_path = os.path.join(dataBaseDir, "airports.csv")
flights_path = os.path.join(dataBaseDir, "flights.csv")
airport_identifier = os.path.join(dataBaseDir, "L_AIRPORT_ID.csv")

In [145]:
# loading airline data into a data frame: 
airline_df = pd.read_csv(airline_path)

In [146]:
# loading airport data into a data frame: 
airports_df = pd.read_csv(airports_path)

In [147]:
# loading flight data into a data frame: 
flight_df = pd.read_csv(flights_path, low_memory=False)

In [148]:
airport_id_df = pd.read_csv(airport_identifier)

In [149]:
def normalize_name(name): 
    return re.sub("[|]|[/]|[.]", " ", name.lower()).replace("airport", "").lower()
def name_code_extractor(row): 
    code = int(row["Code"].split(",")[0])
    name = row["Code"].split(":")[1][1:-1] if ":" in row["Code"] else row["Code"].split(",")[1][1:-1]
    key_feature = normalize_name(name)
    return (name, code, key_feature)

In [150]:
airport_id_df[["AIRPORT_NAME", "AIRPORT_ID", "AIRPORT_KEY_NAME_FEATURES"]] = airport_id_df.apply(name_code_extractor,  axis=1, result_type="expand")

In [151]:
airport_id_df[airport_id_df["AIRPORT_ID"]==14747]

,Code,Description,AIRPORT_NAME,AIRPORT_ID,AIRPORT_KEY_NAME_FEATURES
4525,"14747,""Seattle, WA: Seattle/Tacoma International""",NaN,Seattle/Tacoma International,14747,seattle tacoma international


In [152]:
origin_not_labeled_airports = set(pd.to_numeric(flight_df["ORIGIN_AIRPORT"], errors='coerce').dropna())
destination_not_labeled_airports = set(pd.to_numeric(flight_df["DESTINATION_AIRPORT"], errors='coerce').dropna())
print(f"origin len: {len(origin_not_labeled_airports)}, destination len: {len(destination_not_labeled_airports)}")

origin len: 306, destination len: 307


In [153]:
united_non_labeled_airports = origin_not_labeled_airports.union(destination_not_labeled_airports)
print(f"united_airport: {len(united_non_labeled_airports)}, intersection: {len(origin_not_labeled_airports.intersection(destination_not_labeled_airports))}")

united_airport: 307, intersection: 306


In [154]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def find_airport_best_match(name):
         return process.extractOne(name, airports_df.AIRPORT.tolist())[0]


In [155]:
airport_id_df["BEST_MATCH"] = np.nan

In [156]:
airport_intiials = airport_id_df[airport_id_df["AIRPORT_ID"].isin(united_non_labeled_airports)]["AIRPORT_NAME"].apply(find_airport_best_match)


In [157]:
airport_id_df["BEST_MATCH"] = airport_intiials

In [158]:
flight_df["TRACKED_ORIGIN_LABELS"] = pd.to_numeric(flight_df["ORIGIN_AIRPORT"], errors='coerce')
flight_df["TRACKED_DESTINATION_LABELS"] = pd.to_numeric(flight_df["DESTINATION_AIRPORT"], errors='coerce')

In [159]:
airport_df_with_initials = airport_id_df[airport_id_df["BEST_MATCH"].notnull()].merge(airports_df[["IATA_CODE", "AIRPORT"]], left_on="BEST_MATCH", right_on="AIRPORT", how="left")


In [160]:
flight_df=flight_df.merge(airport_df_with_initials[["AIRPORT_ID", "IATA_CODE"]], left_on="TRACKED_ORIGIN_LABELS", right_on="AIRPORT_ID", how="left")
flight_df.rename(columns={"IATA_CODE": "IATA_CODE_ORIGIN", "AIRPORT_ID": "AIRPORT_ID_ORIGIN"}, inplace=True)

In [161]:
flight_df = flight_df.merge(airport_df_with_initials[["AIRPORT_ID", "IATA_CODE"]], left_on="TRACKED_DESTINATION_LABELS", right_on="AIRPORT_ID", how="left")
flight_df.rename(columns={"IATA_CODE": "IATA_CODE_DESTINATION", "AIRPORT_ID": "AIRPORT_ID_DESTINATION"}, inplace=True)

In [162]:
flight_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,TRACKED_ORIGIN_LABELS,TRACKED_DESTINATION_LABELS,AIRPORT_ID_ORIGIN,IATA_CODE_ORIGIN,AIRPORT_ID_DESTINATION,IATA_CODE_DESTINATION
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
flight_df.loc[pd.to_numeric(flight_df["ORIGIN_AIRPORT"], errors="coerce").notnull(), "ORIGIN_AIRPORT"] = np.nan

In [166]:
flight_df["ORIGIN_AIRPORT"] = flight_df["ORIGIN_AIRPORT"].combine_first(flight_df["IATA_CODE_ORIGIN"])

In [168]:
flight_df.loc[pd.to_numeric(flight_df["DESTINATION_AIRPORT"], errors="coerce").notnull(), "DESTINATION_AIRPORT"] = np.nan
flight_df["DESTINATION_AIRPORT"] = flight_df["DESTINATION_AIRPORT"].combine_first(flight_df["IATA_CODE_DESTINATION"])

## Printing information about tables: 
### Airlines

In [110]:
airline_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   IATA_CODE  14 non-null     object
 1   AIRLINE    14 non-null     object
dtypes: object(2)
memory usage: 352.0+ bytes


As we can see here there are only 14 airlines in the table and no null values.

### Airport Table

In [111]:
airports_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   IATA_CODE  322 non-null    object 
 1   AIRPORT    322 non-null    object 
 2   CITY       322 non-null    object 
 3   STATE      322 non-null    object 
 4   COUNTRY    322 non-null    object 
 5   LATITUDE   319 non-null    float64
 6   LONGITUDE  319 non-null    float64
dtypes: float64(2), object(5)
memory usage: 17.7+ KB


In this case we can see 322 entries in the airport tables, where Longitude and latitude have both 3 entries missing. 

### Flight Table

In [112]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

## Checking missing data on flight Table: 
In order to check the amount of data missing on the flight table, as the amount of columns and rows are much bigger we will check the percentage of data missing per column. 

In [113]:
missing_data = 100*flight_df.isna().sum(axis=0)/len(flight_df)

As we can see here there are some features with a high missing rate, meanwhile there are others with a bigger missing rate. In order to handle this we will drop the rows that are missing in the attributes that have less than 5% missing and in the case of highly missing data as weather_delay we will drop the entire column, as the mayority of the data does not have this attribute set.  

In [114]:
column_to_drop = list(missing_data[missing_data>10].index)

In [115]:
flight_df = flight_df.drop(columns=column_to_drop)

After dropping the data that was missing from the dataset we are going to drop the rows that do not have all the needes data. 

In [116]:
100*flight_df.isna().sum(axis=0)/len(flight_df)

YEAR                   0.000000
MONTH                  0.000000
DAY                    0.000000
DAY_OF_WEEK            0.000000
AIRLINE                0.000000
FLIGHT_NUMBER          0.000000
TAIL_NUMBER            0.252978
ORIGIN_AIRPORT         0.000000
DESTINATION_AIRPORT    0.000000
SCHEDULED_DEPARTURE    0.000000
DEPARTURE_TIME         1.480526
DEPARTURE_DELAY        1.480526
TAXI_OUT               1.530259
WHEELS_OFF             1.530259
SCHEDULED_TIME         0.000103
ELAPSED_TIME           1.805629
AIR_TIME               1.805629
DISTANCE               0.000000
WHEELS_ON              1.589822
TAXI_IN                1.589822
SCHEDULED_ARRIVAL      0.000000
ARRIVAL_TIME           1.589822
ARRIVAL_DELAY          1.805629
DIVERTED               0.000000
CANCELLED              0.000000
dtype: float64

In [117]:
print(f"previous to drop length {len(flight_df)}")

previous to drop length 5819079


In [118]:
flight_df = flight_df.dropna(axis=0)

In [119]:
print(f"post drop length {len(flight_df)}")

post drop length 5714008


After the dropping of rows the amount of data reduced in the table is around 10%. As we still have roughly 57000 rows, we still can train models that can learn without those entries. 

## Uniting the Dataset: 
In order to do this we first need to obtain the longitude and latitude of the airports that are missing in the dataset: 

In [120]:
missing_spacial_info = airports_df[airports_df['LATITUDE'].isnull() | airports_df['LONGITUDE'].isnull()]

In [121]:
missing_spacial_info

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
96,ECP,Northwest Florida Beaches International Airport,Panama City,FL,USA,NaN,NaN
234,PBG,Plattsburgh International Airport,Plattsburgh,NY,USA,NaN,NaN
313,UST,Northeast Florida Regional Airport (St. August...,St. Augustine,FL,USA,NaN,NaN


In [122]:
from geopy.geocoders import Nominatim 

In [123]:
geolocator = Nominatim(user_agent="EDA")
def geocode_airport(row):
    if pd.isna(row['AIRPORT']):
        raise ValueError("There's a row that does not have an airport")
    
    location = geolocator.geocode(row['AIRPORT'].split("(")[0])
    if location:
        return location.latitude, location.longitude

In [124]:
latitude_longitude = missing_spacial_info.apply(geocode_airport, axis=1)

In [125]:
latitude_longitude

96           (30.35993085, -85.80404564346239)
234    (44.651686600000005, -73.4643666977021)
313           (29.96107765, -81.3405011298961)
dtype: object

In [126]:
airports_df.loc[missing_spacial_info.index, ['LATITUDE', 'LONGITUDE']] = list(latitude_longitude)

In [127]:
np.any(flight_df["ORIGIN_AIRPORT"]==14747)

False

In [134]:
sum([value for key, value in flight_df["ORIGIN_AIRPORT"].value_counts().items() if key.isnumeric()])

482878

### Unitind airport information: 
After fixing the latitude and longitude of the other airports we are going to merge the tables into just one table in order to consider all the available data. 

In [91]:
flight_df = flight_df.merge(airports_df[["IATA_CODE", "LATITUDE", "LONGITUDE"]], left_on="ORIGIN_AIRPORT", right_on="IATA_CODE", how="left")
flight_df.rename(columns={"LATITUDE": "ORIGIN_AIRPORT_LATITUDE", "LONGITUDE":"ORIGIN_AIRPORT_LONGITUDE"}, inplace=True)

In [92]:
flight_df = flight_df.merge(airports_df[["IATA_CODE", "LATITUDE", "LONGITUDE"]], left_on="DESTINATION_AIRPORT", right_on="IATA_CODE", how="left")
flight_df.rename(columns={"LATITUDE": "DESTINATION_AIRPORT_LATITUDE", "LONGITUDE":"DESTINATION_AIRPORT_LONGITUDE"}, inplace=True)

In [94]:
flight_df.drop(['IATA_CODE_x', 'IATA_CODE_y'], axis=1, inplace=True)


In [103]:
flight_df["ORIGIN_AIRPORT"].value_counts()

ATL      343506
ORD      276554
DFW      232647
DEN      193402
LAX      192003
          ...  
13541        11
10165         9
14222         9
13502         6
11503         4
Name: ORIGIN_AIRPORT, Length: 628, dtype: int64

In [98]:
airports_df[airports_df["IATA_CODE"]==14747]

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE


## Saving Flight data into Data directory: 
As the data he have till now needs to be generated just once in order to do the EDA. We are going to save this file in order to use it in the EDA.ipynb notebook later. 

In [55]:
flight_df.to_csv(os.path.join("Data", "preprocessed.csv"))